## Important Dependencies with their required versions


In [1]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb


## Imports and Setup and Initializing Agent

In [2]:
# from langchain_community.llms import HuggingFaceHub
# from langchain_community.chat_models.huggingface import ChatHuggingFace
import os
import sys
from io import StringIO
import json
from Model import InitializeModel,InitializeGptModel
from GenerateUnitTest.GenerateTest import InitializeTestChain,queryGpt
from GenerateUnitTest.Feedback import *
from utils.PreprocessUtils import *
from utils.FeedbackUtils import *
from utils.LLMUtilis import *
from utils.FuncUtils import *
from DB import *
from dotenv import load_dotenv
load_dotenv()
# from huggingface_hub import notebook_login
# notebook_login()
import pandas as pd    

HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
from CustomThread import CustomThread

def runCode(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    oldstderr = sys.stderr
    redirected_output2 = sys.stderr = StringIO()
    result = ""
    thread = CustomThread()
    try:
        thread.start()
        exec(code, globals())
        result = redirected_output.getvalue()
    except Exception as e:
        # print(repr(e))
        result = repr(e)
    except SystemExit as s:
        # print(repr(s))
        result = redirected_output2.getvalue()
    except KeyboardInterrupt as k:
        result = "timed out"
    thread.stop()
    globals().popitem()
    sys.stdout = old_stdout
    sys.stderr = oldstderr
    # print(result)
    return result
#########################
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     GPTNeoForCausalLM,
# )
# from tokenizers.implementations import ByteLevelBPETokenizer
# tokenizer = ByteLevelBPETokenizer(
#     "vocab.json",
#     "merges.txt",
# )
load_dotenv()
model_id="google/gemma-7b-it"
model_id="mistralai/Mixtral-8x7B-Instruct-v0.1"
max_new_tokens= 6000
if 'Mixtral' in model_id:
    max_new_tokens= 31_000

llm, chat_model = InitializeModel(os.environ['HF_TOKEN'], model_id,
                                  max_new_tokens= max_new_tokens)
if 'Mixtral' in model_id:
    GenUnitTestChain= InitializeTestChain(llm,True)
else:
    GenUnitTestChain= InitializeTestChain(chat_model,True)

if 'Mixtral' in model_id:
    UnitTestFeedbackChain= InitializeFeedbackChain(llm)
else:
    UnitTestFeedbackChain= InitializeFeedbackChain(chat_model)
    
HEval_JsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
db= connect_db()
# llm2,llm3=InitializeGptModel(os.environ['OPENAI_API_KEY'])


C:\Users\omare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
C:\Users\omare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you in

## Generate Unit Tests from Code

In [ ]:
total_test_cases = 0
total_success_test_cases = 0
failed_test_cases = 0
error_test_cases = 0
num_failed_examples = 0
num_successful_examples = 0
testsToRepeat = 0     # Number of tests to repeat = Summation of all tests under failed examples whether they are failed or succeeded
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]
codeTobeRunList = []
TotalCases=0
TotalFailedCases=0
incompleteResponses=0
OutputFile="OutputTest/"
JSONFile=OutputFile+"RunningLogs.json"
CasesJSONFile=OutputFile+"Cases.json"

#check if directory exists
if not os.path.exists(OutputFile):
  try:
    os.makedirs(OutputFile)
    print(f"Directory {OutputFile} created successfully!")
  except Exception as e:
    print(f"Error creating directory {OutputFile}: {e}")
    exit()
  
if not os.path.exists(CasesJSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(CasesJSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {CasesJSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {CasesJSONFile}: {e}")
    exit()

if not os.path.exists(JSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(JSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {JSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {JSONFile}: {e}")
    exit()

df=pd.DataFrame()
casesDf=pd.DataFrame()
for i in range(0,30):
    # if (i == 10): continue
    print("Running Test Case ",i)
    with open (OutputFile+'Cases.txt','a') as c:
      c.write("Running Test Case "+str(i)+"\n=====================================\n")
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton, utility =get_function_name(funcDefiniton)
    code = utility + "\n" + funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    codes_of_few_shot,test_cases_few_shot=get_few_shots(db,code) 
    #take the most similar few shot other than the code itself
    codes_of_few_shot,test_cases_few_shot=codes_of_few_shot[1:2],test_cases_few_shot[1:2]
    fewShotStr = preprocessStringFewShot(codes_of_few_shot, test_cases_few_shot)
    # print("fewShotStr is :",fewShotStr)
    try:
        unittest=GenUnitTestChain.invoke({"description":description,"code":code,"test_cases_of_few_shot":fewShotStr})#,"test_cases_of_few_shot":fewShotStr
    except Exception as e:
        print("ERROR in invoking GenUnitTestChain")
        print(e)
        with open (OutputFile+'cases.txt','a') as c:
          c.write("Test Case "+str(i)+" Didn't Run Due to Errorr\n=====================================\n")
        continue
    ############################################

    ### get the response (unit test) code only without natural language
    unittest_code,isIncompleteResponse = get_code_from_response(unittest['text'])
    if(isIncompleteResponse):
        incompleteResponses+=1
        print("Test Case "+str(i)+" Didn't Run Due to Incomplete Response\n=====================================\n")
        with open (OutputFile+'cases.txt','a') as c:
          c.write("Test Case "+str(i)+" Didn't Run Due to Incomplete Response\n=====================================\n")

    #print("unittest_code is ",unittest_code)
    unittest_code = preprocessUnitTest(unittest_code)
    # codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    codeTobeRun = get_running_code(code,unittest_code)
    # print("unittest_code is: \n ",unittest_code)
    # print("run_code is \n",codeTobeRun)
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    # print("Feedback is ", feedback)
    
    if(feedbackparsed=='' or feedbackparsed is None):
        #in case of OK
        num_successful_examples +=1
        OKCases+=1
        print(f"Test example {i} succeeded\n======================================\n" )
        num_of_assertions = get_num_assertions(unittest_code)
        print("Number of Ran Tests : " , num_of_assertions) 
        print("Number of Succeeded Test : ", num_of_assertions)
        print("Number of Succeeded Test : ", 0)
        with open (OutputFile+'cases.txt','a') as c:
          c.write("Test example "+str(i)+" succeeded\n")
          c.write("Number of Ran Tests : "+str(num_of_assertions)+'\n')
          c.write("Number of Succeeded Test : "+str(num_of_assertions)+'\n')
        new_case_row=pd.DataFrame({'CaseNumber':i,'Total Tests':num_of_assertions,'Tests failed':0},index=[0])
        casesDf=pd.concat([casesDf,new_case_row])
        casejsondata=casesDf.to_dict(orient='records')
        with open(CasesJSONFile, 'w') as f:
            json.dump(casejsondata, f, indent=4)
        
        total_test_cases += num_of_assertions

    else:
         num_failed_examples +=1
         total_cases, failed_cases, error_cases = get_failed_testcases(feedback)
         print(f"Test example {i} failed\n======================================\n" )
         print("Number of Ran Tests : " , total_cases) 
         print("Number of failed Tests : ", failed_cases)
         print("Number of Succeeded Test : ", total_cases - failed_cases)
         with open (OutputFile+'cases.txt','a') as c:
          c.write("Test example "+str(i)+" failed\n")
          c.write("Number of Ran Tests : "+str(total_cases)+'\n')
          c.write("Number of failed Tests : "+str(failed_cases)+'\n')
          c.write("Number of Succeeded Test : "+str(total_cases - failed_cases)+'\n')
         new_case_row=pd.DataFrame({'CaseNumber':i,'Total Tests':total_cases,'Tests failed':failed_cases},index=[0])
         casesDf=pd.concat([casesDf,new_case_row])
         casejsondata=casesDf.to_dict(orient='records')
         with open(CasesJSONFile, 'w') as f:
             json.dump(casejsondata, f, indent=4)
         testsToRepeat += total_cases
         total_test_cases += total_cases
         failed_test_cases+= failed_cases
        

    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    new_row=pd.DataFrame({'CaseNumber':i,'Description':description,'Code':code,'GeneratedCode':unittest_code,'Feedback':feedbackparsed,"FullFeedback":feedback},index=[0])
    # df = df.append(new_row, ignore_index=True)
    df=pd.concat([df,new_row])
    # Save the updated DataFrame back to the excel file using 'openpyxl' engine for writing
    jsondata=df.to_dict(orient='records')
    with open(JSONFile, 'w') as f:
        json.dump(jsondata, f, indent=4)

#dump data into xlsx file
# resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
# #resDF.to_excel("GeneratedFeedback.xlsx",index=False)
# resDF.to_json("GeneratedFeedback.json",orient='records')
# print("OKCases are ",OKCases)
print('\n End of Test Cases \n')
print("Total succeeded examples : " ,num_successful_examples)
print("Total failed examples : ", num_failed_examples)
print("Total  testcases : " ,total_test_cases)
print("Total succeeded testcases : ",total_test_cases- failed_test_cases-incompleteResponses)
print("Total failed testcases : ",failed_test_cases)
print("Tests to repeat : ",testsToRepeat)
with open(OutputFile+'Res.txt', 'w') as f:
    f.write('Total succeeded examples : ' + str(num_successful_examples) + '\n')
    f.write('Total failed examples : ' + str(num_failed_examples) + '\n')
    f.write('Total  testcases : ' + str(total_test_cases) + '\n')
    f.write('Total succeeded testcases : ' + str(total_test_cases- failed_test_cases-incompleteResponses) + '\n')
    f.write('Total failed testcases : ' + str(failed_test_cases) + '\n')
    f.write('Incomplete Responses are : ' + str(incompleteResponses) + '\n')
    f.write('Tests to repeat : ' + str(testsToRepeat) + '\n')

print("Incomplete Responses are ",incompleteResponses)

Running Test Case  0
Test example 0 succeeded

Number of Ran Tests :  7
Number of Succeeded Test :  7
Number of Succeeded Test :  0
Running Test Case  1
Test example 1 succeeded

Number of Ran Tests :  0
Number of Succeeded Test :  0
Number of Succeeded Test :  0
Running Test Case  2
Test Case 2 Didn't Run Due to Incomplete Response

Test example 2 failed

Number of Ran Tests :  0
Number of failed Tests :  0
Number of Succeeded Test :  0
Running Test Case  3
Test example 3 failed

Number of Ran Tests :  7
Number of failed Tests :  2
Number of Succeeded Test :  5
Running Test Case  4
Test example 4 failed

Number of Ran Tests :  7
Number of failed Tests :  2
Number of Succeeded Test :  5
Running Test Case  5
Test example 5 failed

Number of Ran Tests :  7
Number of failed Tests :  1
Number of Succeeded Test :  6
Running Test Case  6
Test example 6 failed

Number of Ran Tests :  7
Number of failed Tests :  2
Number of Succeeded Test :  5
Running Test Case  7
Test example 7 failed

Number

## Feedback to LLM for regenerating again

In [5]:
total_test_cases = 0
total_success_test_cases = 0
failed_test_cases = 0
num_failed_examples = 0
num_successful_examples = 0
testsToRepeat = 0     # Number of tests to repeat = Summation of all tests under failed examples whether they are failed or succeeded
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]
TotalCases=0
TotalFailedCases=0
incompleteResponses=0
OutputFile="Output/"
JSONFile=OutputFile+"RunningLogs.json"
CasesJSONFile=OutputFile+"Cases.json"
OldFile="Results/Gemma-0Shot/"
OldCasesFile= OldFile+"Cases.json"
OldJsonFile= OldFile+"RunningLogs.json"
CasesLogs=pd.read_json(OldJsonFile)
OldCases= pd.read_json(OldCasesFile)

if not os.path.exists(OutputFile):
  try:
    os.makedirs(OutputFile)
    print(f"Directory {OutputFile} created successfully!")
  except Exception as e:
    print(f"Error creating directory {OutputFile}: {e}")
    exit()
if not os.path.exists(JSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(JSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {JSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {JSONFile}: {e}")
    exit()
# try:


if not os.path.exists(CasesJSONFile):
  try:
    # Create a new empty JSON file with an empty list structure
    with open(CasesJSONFile, 'w') as f:
      json.dump([], f)
    print(f"File {CasesJSONFile} created successfully!")
  except Exception as e:
    print(f"Error creating file {CasesJSONFile}: {e}")
    exit()

df=pd.DataFrame()
casesDf=pd.DataFrame()

for i in range(len(CasesLogs)):
    currDescription=CasesLogs.iloc[i]['Description']
    currCode=CasesLogs.iloc[i]['Code']
    currGeneratedCode=CasesLogs.iloc[i]['GeneratedCode']
    currFeedback=CasesLogs.iloc[i]['Feedback']
    print("Running Test Case ",i)
    # no feedback means testcase passed so don't run it again
    print("Curr Feedback is ", currFeedback)
    if pd.isna(currFeedback) or currFeedback=="" or currFeedback is None:
       print("Case",i," is already passed")
       with open (OutputFile+'Cases.txt','a') as c:
        c.write("Case "+str(i)+" is already passed\n=====================================\n")
       continue
    try:
        GenerationPostFeedback=UnitTestFeedbackChain.invoke({"description":currDescription,"code":currCode,"UnitTests":currGeneratedCode,"Feedback":currFeedback})
        print("Full Generation is :",GenerationPostFeedback['text'])
    except Exception as e:
        print("ERROR in invoking UnitTestFeedbackChain")
        print(e)
        with open (OutputFile+'Cases.txt','a') as c:
          c.write("Test Case "+str(i)+" Didn't Run Due to Errorr\n=====================================\n")
        continue
    newUnitTestCode,isIncompleteResponse = get_code_from_response(GenerationPostFeedback['text'])
    if(isIncompleteResponse):
        incompleteResponses+=1
        print("Test Case "+str(i)+" Didn't Run Due to Incomplete Response\n=====================================\n")
        with open (OutputFile+'cases.txt','a') as c:
          c.write("Test Case "+str(i)+" Didn't Run Due to Incomplete Response\n=====================================\n")

    unittest_code = preprocessUnitTest(newUnitTestCode)
    codeTobeRun = get_running_code(currCode,unittest_code)
    feedback = runCode(code=codeTobeRun)
    print("original feedback is: ",feedback)
    feedbackparsed=get_feedback_from_run(feedback)

    if(feedbackparsed==''):
        #in case of OK
        num_successful_examples +=1
        OKCases+=1
        print(f"Test example {i} succeeded" )
        num_of_assertions = get_num_assertions(unittest_code)
        print("Number of Ran Tests : " , num_of_assertions) 
        print("Number of Succeeded Test : ", num_of_assertions)
        # print("Number of Succeeded Test : ", 0)
        with open (OutputFile+'cases.txt','a') as c:
          c.write("Test example "+str(i)+" succeeded\n")
          c.write("Number of Ran Tests : "+str(num_of_assertions)+'\n')
          c.write("Number of Succeeded Test : "+str(num_of_assertions)+'\n')
        
        oldTotalTests=OldCases.iloc[i]['Total Tests']
        oldTestsFailed=OldCases.iloc[i]['Tests failed']

        new_case_row=pd.DataFrame({'CaseNumber':i,'Feedback Total Tests':num_of_assertions,'Feedback Tests failed':0,
                                    'Old Total Tests': oldTotalTests, 'Old Tests Failed': oldTestsFailed },index=[0])
        casesDf=pd.concat([casesDf,new_case_row])
        casejsondata=casesDf.to_dict(orient='records')
        with open(CasesJSONFile, 'w') as f:
            json.dump(casejsondata, f, indent=4)

        total_test_cases += num_of_assertions

    else:
         num_failed_examples +=1
         total_cases, failed_cases, error_cases = get_failed_testcases(feedback)
         print(f"Test example {i} failed" )
         print("Number of Ran Tests : " , total_cases) 
         print("Number of failed Tests : ", failed_cases)
         print("Number of Succeeded Test : ", total_cases - failed_cases)
         with open (OutputFile+'cases.txt','a') as c:
          c.write("Test example "+str(i)+" failed\n")
          c.write("Number of Ran Tests : "+str(total_cases)+'\n')
          c.write("Number of failed Tests : "+str(failed_cases)+'\n')
          c.write("Number of Succeeded Test : "+str(total_cases - failed_cases)+'\n')
         
         oldTotalTests=OldCases.iloc[i]['Total Tests']
         oldTestsFailed=OldCases.iloc[i]['Tests failed']
         new_case_row=pd.DataFrame({'CaseNumber':i,'Feedback Total Tests':total_cases,'Feedback Tests failed':failed_cases,
                                    'Old Total Tests': oldTotalTests, 'Old Tests Failed': oldTestsFailed },index=[0])
         casesDf=pd.concat([casesDf,new_case_row])
         casejsondata=casesDf.to_dict(orient='records')
         with open(CasesJSONFile, 'w') as f:
             json.dump(casejsondata, f, indent=4)
         testsToRepeat += total_cases 
         total_test_cases += total_cases
         failed_test_cases+= failed_cases
    
    descriptions.append(currDescription)
    codes.append(currCode)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    new_row=pd.DataFrame({'CaseNumber':i,'Description':currDescription,'Code':currCode,'GeneratedCode':unittest_code,'Feedback':feedbackparsed},index=[0])
    df = df._append(new_row, ignore_index=True)
    # Save the updated DataFrame back to the excel file using 'openpyxl' engine for writing
    jsondata=df.to_dict(orient='records')
    with open(JSONFile, 'w') as f:
        json.dump(jsondata, f, indent=4)


c.close()
print('\n End of Test Cases \n')
print("Total succeeded examples : " ,num_successful_examples)
print("Total failed examples : ", num_failed_examples)
print("Total  testcases : " ,total_test_cases)
print("Total succeeded testcases : ",total_test_cases- failed_test_cases-incompleteResponses)
print("Total failed testcases : ",failed_test_cases)
print("Tests to repeat : ",testsToRepeat)
with open(OutputFile+'Res.txt', 'w') as f:
    f.write('Total succeeded examples : ' + str(num_successful_examples) + '\n')
    f.write('Total failed examples : ' + str(num_failed_examples) + '\n')
    f.write('Total  testcases : ' + str(total_test_cases) + '\n')
    f.write('Total succeeded testcases : ' + str(total_test_cases- failed_test_cases-incompleteResponses) + '\n')
    f.write('Total failed testcases : ' + str(failed_test_cases) + '\n')
    f.write('Tests to repeat : ' + str(testsToRepeat) + '\n')
    f.write('Incomplete Responses are : ' + str(incompleteResponses) + '\n')

print("Incomplete Responses are ",incompleteResponses)


File Output/RunningLogs.json created successfully!
File Output/Cases.json created successfully!
Running Test Case  0
Curr Feedback is  ERROR: test_has_close_elements_with_non_numeric_elements (__main__.HasCloseElementsTest)
Traceback (most recent call last):
  File "<string>", line 55, in test_has_close_elements_with_non_numeric_elements
  File "<string>", line 13, in has_close_elements
TypeError: unsupported operand type(s) for -: 'float' and 'str'


Full Generation is : <s> [INST] You are a Python expert, and your task is to debug and improve the following Python code based on the given description, code snippet, and unit tests feedback:

*Description:*
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True

*Given Code:*
```python
from typing import List


def has_close_elements(numbers: List[float], th

## GPT MODEL

In [ ]:
introCode = """
import sys
sys.setrecursionlimit(100)
"""
import time
OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]

for i in range(0,10):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=HEval_JsonObj.iloc[i]['text']
    code=HEval_JsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=HEval_JsonObj.iloc[i]['prompt']
    funcDefiniton=get_function_name(funcDefiniton)
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    code_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessStringFewShot(code_of_few_shot, test_cases_few_shot)

    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model
    #try gpt model
    unittest=queryGpt(llm2,description,code,fewshots=False,test_cases_of_few_shot=fewShotStr)['choices'][0]['message']['content']
    print("gpt_response is ",unittest)

    
    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$  karim gpt model

    ### get the response (unit test) code only without natural language
    unittest_code = get_code_from_response(unittest,funcDefiniton)
    unittest_code = replaceUnitTestCall(unittest_code)
    ##############run code with the original and unit test concatenated #######################
    codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    print("run_code is")
    print(codeTobeRun)
    # print("------------------------------lolooloolololo ------------------------------------")
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    if(feedbackparsed==''):
        #in case of OK
        OKCases+=1
        # feedbackparsed=""
        # for i,line in enumerate(feedback.split('\n')):
        #     if i>1:
        #         feedbackparsed+=line+'\n'
    #print(feedbackparsed)
    print(feedback)
    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed
    time.sleep(20)

#dump data into xlsx file
resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
#resDF.to_excel("GeneratedFeedbackGPT.xlsx",index=False)
resDF.to_json("GeneratedFeedbackGPT.json",orient='records')
print("OKCases are ",OKCases)
print("Total Cases are ",len(descriptions))
with open('ResGPT.txt', 'w') as f:
    f.write("OKCases are "+str(OKCases)+"\n")
    f.write("Total Cases are "+str(len(descriptions))+"\n")


## Bug Fixing

In [ ]:
BugRepair_template_str = """Act as a python expert in bug fixing and assume that I need to fix all errors perfectly.
I am going to give you the bugs encountered due to running test cases then after
that I will give you the code under test with bugs so that you fix it, the line numbers of
the error correspond to the line that the error occured in the code provided.
The following is the code under test:
```python
{code}```

The following are the bugs and test cases corresponding to the bugs:
```python
{TestcaseCode}```

The result after running these test cases on an interpreter:
{InterpreterFeedback}

Change either the code or the test cases according to the feedback of the interpreter.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
"""
InterpreterFeedback=feedbackparsed

BugRepair_Template = PromptTemplate(
    template=BugRepair_template_str,
    input_variables=["code","TestcaseCode","InterpreterFeedback"],
    verbose=True
    )

BugFixChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=BugRepair_Template
)

res=BugFixChain.invoke({
                             "code":code,
                             "TestcaseCode":res_code,
                             "InterpreterFeedback":InterpreterFeedback
                             })
print(res['text'])

## input code preprocessor

In [ ]:
#if the input code is without description, we want to generate description
Generate_description_Template = """Act as a python expert, Describe the purpose and functionality of the following Python code.

```python
{code}```
The output should be like that:
The provided Python code aims to [explain purpose].
In summary, [concise summary of functionality ].
Examples: [3 or 4 input output examples]
"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
Generate_description_PromptTemplate = PromptTemplate(
    template=Generate_description_Template,
    input_variables=["code"],
    verbose=True
    )

GenDescriptionChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_description_PromptTemplate
)
res=GenDescriptionChain.invoke({
                             "code":code
                             })
print(res['text'])